# Задача 2

## Вариант 5

$$
f_\theta(x) = \frac{1}{(k-1)! \, \theta^k} \, x^{k-1} e^{-x/\theta} \cdot \mathbf{1}(x > 0),
$$

### Нахождение оценки

Методом моментов найти оценку параметра $\theta$, если $k \in \mathbb{N}$ – известный параметр

Заметим!, что распределение является частным случаем Гамма-распределения – распределением Эрланга. Характеристики данного распределения:
$$
\mathbb{E}[\mathbb{X}] = k \theta \quad \mathbb{D}[\mathbb{X}] = k \theta^2
$$

Пусть есть выборка $X_1, X_2 \dots X_n$. Ее среднее равно $\overline{X}=\sum^n_{i=1}X_i$. Тогда получим оценку параметра $\theta$:
$$
\hat{\theta}=\frac{\overline{X}}{k}
$$

### Смещение

Найдем смещение:
$$
\operatorname{Bias}(\hat{\theta_n})=\mathbb{E}[\hat{\theta_n}]-\theta
$$

$$
\hat{\theta_n}=\frac{\overline{X}_n}{k}=\frac{1}{kn}\sum^n_{i=1}X_i,\quad X_i \sim \operatorname{Gamma}(k, \theta)
$$

Зная что чем является в гамма-распределении получаем:
$$
\mathbb{E}[\hat{\theta_n}]=\frac{1}{kn}\sum^n_{i=1} \mathbb{E}[X_i]=\frac{1}{k}\cdot k \theta=\theta
$$

Оценка получилась несмещенной. $\operatorname{Bias} = 0$.

### Дисперсия

Найдем дисперсию:
$$
\mathbb{D}[\hat{\theta_n}]=\mathbb{D}[\frac{\overline{X}_n}{k}]=\frac{1}{k^2} \mathbb{D}[\overline{X}_n]=\frac{1}{k^2} \frac{1}{n} \mathbb{D}[X_1]
$$

Зная теоретическую оценку дисперсии гамма-распределения, получаем:
$$
\mathbb{D}[\hat{\theta_n}]=\frac{1}{k^2} \frac{1}{n} \theta^2 k = \frac{\theta^2}{kn}
$$

### Среднеквадратическая ошибка

Оценка является несмещенной, а для несмещенных оценок известно, что MSE совпадает с дисперсией. Делаем из этого логический вывод:
$$
\operatorname{MSE}(\hat{\theta}_n)=\frac{\theta^2}{kn}
$$

### Состоятельность оценки

По неравенству Чебышева и ЗБЧ:
$$
P(|\hat{\theta_n}-\theta|\le \varepsilon) \ge \frac{\mathbb{D}[\hat{\theta_n}]}{n \varepsilon^2} = \frac{\theta^2}{kn \varepsilon^2} \to 0
$$

Оценка состоятельная

### Асимптотическая нормальность

Докажем, что при $n \to \infty$:
$$
\sqrt{n}(\hat{\theta}_n-\theta) \to N(0, \sigma^2)
$$

По ЦПТ плюс-минус очевидно, показываем, что теорема работает для $\overline{X}_n$, затем выражаем оценку через $X$, получаем, что действительно данные выражени сходится к $N(0, \frac{\theta^2}{k})$

In [57]:
import numpy as np
import plotly.express as px

In [58]:
n_values = [10, 100, 1000, 10000, 50000]
m = 1000

In [59]:
def calculate_theta(x: np.array, k: int) -> float:
    return np.mean(x) / k

In [60]:
theta = 2
k = 3
epsilon = 0.5

In [61]:
import numpy as np

class Report:
    n: int
    predicted_thetas: np.array
    biases: np.array
    variances: np.array
    MSE: np.array
    ratio: np.array

    def generate_report(self, true_theta=3.0, k=2, epsilon=0.5) -> str:
        m = len(self.predicted_thetas)
        mean_theta = np.mean(self.predicted_thetas)
        bias = mean_theta - true_theta
        exp_variance = np.var(self.predicted_thetas, ddof=1)
        mse = np.mean((self.predicted_thetas - true_theta)**2)
        exceed_ratio = np.mean(np.abs(self.predicted_thetas - true_theta) > epsilon)

        theo_variance = true_theta**2 / (k * self.n)
        theo_mse = theo_variance

        def fmt(x, decimals=4):
            if abs(x) < 1e-4 or abs(x) > 1e4:
                return f"{x:.3e}"
            return f"{x:.{decimals}f}"

        report = f"""
{'═' * 80}
📊 СТАТИСТИЧЕСКИЙ ОТЧЕТ ДЛЯ n = {self.n}
{'═' * 80}
🎯 Истинное θ = {true_theta}, k = {k}, ε = {epsilon}, симуляций = {m}

{'─' * 80}
⚖️ СРАВНЕНИЕ РЕЗУЛЬТАТОВ
┌───────────────────────┬───────────────────┬───────────────────┬──────────────┐
│ Параметр              │ Эксперимент       │ Теория            │ Отклонение   │
├───────────────────────┼───────────────────┼───────────────────┼──────────────┤
│ Смещение (Bias)       │ {fmt(bias):<17} │ 0.0000            │ {abs(bias)/true_theta*100:.2f}% │
│ Дисперсия             │ {fmt(exp_variance):<17} │ {fmt(theo_variance):<17} │ {abs(exp_variance-theo_variance)/theo_variance*100:.2f}% │
│ MSE                   │ {fmt(mse):<17} │ {fmt(theo_mse):<17} │ {abs(mse-theo_mse)/theo_mse*100:.2f}% │
│ |θ̂-θ| > {epsilon}          │ {fmt(exceed_ratio):<17} │ -                 │ -            │
└───────────────────────┴───────────────────┴───────────────────┴──────────────┘

{'─' * 80}
🔍 ВЫВОДЫ:
• Несмещённость: {'✓' if abs(bias) < 0.01 else '✗'} Смещение = {fmt(bias)} (порог 0.01)
• Эффективность: {'✓' if abs(exp_variance-theo_variance)/theo_variance < 0.05 else '✗'} Отклонение дисперсии = {abs(exp_variance-theo_variance)/theo_variance*100:.1f}% (порог 5%)
• Точность: {'✓' if exceed_ratio < 0.05 else '⚠'} {exceed_ratio*100:.1f}% оценок вне диапазона [{true_theta-epsilon}, {true_theta+epsilon}]

{'═' * 80}
💡 РЕКОМЕНДАЦИИ:
─ Для уменьшения ошибок до 1% необходим объем выборки n ≥ {int(true_theta**2/(k*epsilon**2*0.01))}
─ При увеличении k до 3 дисперсия уменьшится на {(k/3)*100:.0f}%
{'═' * 80}
"""
        return report

In [62]:
def generate_report(n: int, m: int):
    report = Report()
    report.n = n

    samples = np.random.gamma(scale=k, shape=3, size=(m, n))

    sample_means = np.mean(samples, axis=1)
    predicted_thetas = sample_means / k
    report.predicted_thetas = predicted_thetas

    biases = predicted_thetas - theta
    report.biases = biases

    variances = predicted_thetas.var(ddof=1)
    report.variances = variances

    mse = np.mean((predicted_thetas - theta)**2)
    report.MSE = mse

    exceedance = np.mean(np.abs(predicted_thetas - theta) > epsilon)
    report.ratio = exceedance

    return report.generate_report()

In [63]:
for n in n_values:
    print(generate_report(n, m))


════════════════════════════════════════════════════════════════════════════════
📊 СТАТИСТИЧЕСКИЙ ОТЧЕТ ДЛЯ n = 10
════════════════════════════════════════════════════════════════════════════════
🎯 Истинное θ = 3.0, k = 2, ε = 0.5, симуляций = 1000

────────────────────────────────────────────────────────────────────────────────
⚖️ СРАВНЕНИЕ РЕЗУЛЬТАТОВ
┌───────────────────────┬───────────────────┬───────────────────┬──────────────┐
│ Параметр              │ Эксперимент       │ Теория            │ Отклонение   │
├───────────────────────┼───────────────────┼───────────────────┼──────────────┤
│ Смещение (Bias)       │ 0.0137            │ 0.0000            │ 0.46% │
│ Дисперсия             │ 0.2892            │ 0.4500            │ 35.73% │
│ MSE                   │ 0.2891            │ 0.4500            │ 35.76% │
│ |θ̂-θ| > 0.5          │ 0.3250            │ -                 │ -            │
└───────────────────────┴───────────────────┴───────────────────┴──────────────┘

─────────────

In [64]:
def generate_histogram(n: int, m: int):
    samples = np.random.gamma(scale=k, shape=3, size=(m, n))
    sample_means = np.mean(samples, axis=1)
    predicted_thetas = sample_means / k

    fig = px.histogram(predicted_thetas, nbins=20, title=f"Выборки с объемом n={n}")
    fig.show()

In [65]:
for n in n_values:
    generate_histogram(n, m)